In [3]:
import librosa  
import numpy as np  
import os

In [4]:
def extract_features(audio_path):
    data, sr = librosa.load(audio_path, res_type='kaiser_fast')
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sr, n_mfcc=128).T, axis=0)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

In [7]:
X = []
Y = []
base_dir = os.getcwd()
emotions = ['Neutral', 'Happy', 'Sad', 'Angry', 'Fearful', 'Disgusted', 'Surprised']

for emotion in emotions:
    emotion_dir = os.path.join(base_dir, emotion)